# Introduction to atomman: Basic support and analysis tools

__Lucas M. Hale__, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), _Materials Science and Engineering Division, NIST_.
    
[Disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm) 

## 1. Introduction<a id='section1'></a>

This Notebook outlines some of the other tools in atomman that provide basic support features and simple analysis of the atomistic systems.

**Library Imports**

In [1]:
# Standard Python libraries
import os
from io import open
from copy import deepcopy
import datetime

# http://www.numpy.org/
import numpy as np

# https://pandas.pydata.org/
import pandas as pd

# https://github.com/usnistgov/atomman
import atomman as am
import atomman.unitconvert as uc

# Show atomman version
print('atomman version =', am.__version__)

# Show date of Notebook execution
print('Notebook executed on', datetime.date.today())

atomman version = 1.5.0
Notebook executed on 2025-02-24


Construct a demonstration 2x2x2 diamond cubic silicon system

In [2]:
a = uc.set_in_units(5.431, 'angstrom')
ucell = am.load('prototype', 'A4--C--dc', a=a, symbols='Si')

system = ucell.supersize(2,2,2)

print(system.natoms)

64


## 2. Elastic constants <a id='section2'></a>

The full elastic constants tensor for a given crystal can be represented with the atomman.ElasticConstants class.  The values in an ElasticConstants object can be set and retrieved in a variety of formats and transformed to other Cartesian coordinate systems. 

See the [3.1. ElasticConstants class Jupyter Notebook](3.1. ElasticConstants class.ipynb) for more details and a full description of all of the class methods.

In [3]:
# Define an ElasticConstants object for diamond cubic silicon
# values taken from http://www.ioffe.ru/SVA/NSM/Semicond/Si/mechanic.html
C11 = uc.set_in_units(16.60 * 10**11, 'dyn/cm^2')
C12 = uc.set_in_units( 6.40 * 10**11, 'dyn/cm^2')
C44 = uc.set_in_units( 7.96 * 10**11, 'dyn/cm^2')

C = am.ElasticConstants(C11=C11, C12=C12, C44=C44)

In [4]:
# Get 6x6 Cij Voigt representation of elastic constants in GPa
print('Cij (GPa) =')
print(uc.get_in_units(C.Cij, 'GPa'))

Cij (GPa) =
[[166.   64.   64.    0.    0.    0. ]
 [ 64.  166.   64.    0.    0.    0. ]
 [ 64.   64.  166.    0.    0.    0. ]
 [  0.    0.    0.   79.6   0.    0. ]
 [  0.    0.    0.    0.   79.6   0. ]
 [  0.    0.    0.    0.    0.   79.6]]


## 3. Relative distances between atoms<a id='section3'></a>

There are a few built-in tools for investigating the relative positions between atoms of the same and different systems.

### 3.1. System.dvect()

The System.dvect() method computes the shortest vector(s) between two points or list of points within the System taking into account the System's periodic dimensions.

Parameters

- __pos_0__ (*numpy.ndarray or index*) Absolute Cartesian vector position(s) to use as reference point(s). If the value can be used as an index, then self.atoms.pos[pos_0] is taken.
- __pos_1__ (*numpy.ndarray or index*) Absolute Cartesian vector position(s) to find relative to pos_0.  If the value can be used as an index, then self.atoms.pos[pos_1] is taken.

In [5]:
# Calculate shortest vector between atoms 1 and 60
print(system.dvect(1, 60))

[ 4.07325  4.07325 -4.07325]


### 3.2. System.dmag()

The System dmag() method computes the magnitude of the shortest vector(s) between two points or list of points within the System taking into account the System's periodic dimensions.  This is identical to computing dvect above, then finding the magnitude of those vectors, but should be faster.

In [6]:
# Calculate shortest distance between position [5., 5., 5.] and all atoms in system
dmags = system.dmag([5.0, 5.0, 5.0], range(system.natoms))
print(dmags)

[8.66025404 5.95297241 5.95297241 5.95297241 6.30856205 3.87088054
 3.87088054 3.87088054 7.08419092 6.33398788 6.33398788 3.25939281
 5.45266877 2.2175116  5.86626957 5.86626957 7.08419092 6.33398788
 3.25939281 6.33398788 5.45266877 5.86626957 2.2175116  5.86626957
 5.03701519 6.69334927 3.91218142 3.91218142 4.43455051 4.93424363
 4.93424363 7.33774633 7.08419092 3.25939281 6.33398788 6.33398788
 5.45266877 5.86626957 5.86626957 2.2175116  5.03701519 3.91218142
 6.69334927 3.91218142 4.43455051 4.93424363 7.33774633 4.93424363
 5.03701519 3.91218142 3.91218142 6.69334927 4.43455051 7.33774633
 4.93424363 4.93424363 0.7465139  4.4706471  4.4706471  4.4706471
 3.09820588 6.6163557  6.6163557  6.6163557 ]


### 3.3. displacement()

The atomman.displacement() function compares two systems with the same number of atoms and calculates the vector differences between all atoms with the same atomic id's. The vectors returned are the shortest vectors after taking periodic boundaries in consideration, i.e. it uses dvect().

Parameters

- __system_0__ (*atomman.System*) The initial system to calculate displacements from.
- __system_1__ (*atomman.System*) The final system to calculate displacements to.
- __box_reference__ (*str or None*) Specifies which system's boundary conditions to use.
    - 'initial' uses system_0's box and pbc.
    - 'final' uses system_1's box and pbc (Default).
    - None computes the straight difference between the positions without accounting for periodic boundaries.

In [7]:
# Copy system and randomly displace atoms
system2 = deepcopy(system)
system2.atoms.pos += 3 * np.random.rand(system.natoms, 3)
system2.wrap()

# Show displacement between the two systems
print(am.displacement(system, system2))       

[[1.68329398e+00 1.59471821e+00 1.12567820e+00]
 [2.01354374e+00 2.80202757e+00 2.07831868e+00]
 [7.47449695e-01 1.30569037e+00 7.67694844e-01]
 [2.53891312e+00 1.99902790e-01 1.76156750e-01]
 [2.18386824e+00 2.12807404e+00 1.12800108e+00]
 [2.59555119e+00 9.23720382e-01 2.89071136e+00]
 [2.30897728e+00 2.03973236e+00 2.46366198e+00]
 [2.90574140e+00 2.86165249e+00 1.10077852e+00]
 [1.80186940e+00 7.85401371e-01 2.27787769e+00]
 [1.86689561e+00 8.65234632e-01 2.23399686e+00]
 [2.89412332e+00 5.19416198e-01 1.26167695e+00]
 [1.57077452e+00 2.05365198e+00 1.33155584e+00]
 [2.05482369e-01 1.54801111e+00 2.24957987e-01]
 [2.57773418e+00 7.46273006e-01 2.31568578e+00]
 [2.57051177e-01 9.60543891e-01 2.57173643e+00]
 [3.57480228e-01 2.63535713e+00 2.12260630e+00]
 [2.05966822e+00 1.32114362e+00 1.86791490e+00]
 [2.55674968e+00 2.72003570e+00 2.56313648e+00]
 [5.16720993e-01 2.87797924e+00 2.70452614e+00]
 [2.40903688e+00 2.64268832e+00 1.56184587e+00]
 [1.65190074e+00 1.27956154e+00 1.395431

### 3.4. System.neighborlist()

A list of neighbor atoms within a cutoff can be constructed using the System.neighborlist() method.  The list of neighbors is returned as an atomman.NeighborList object.

See the [3.2. NeighborList class Jupyter Notebook](3.2. NeighborList class.ipynb) for more details on how the list is calculated and can be used.

Parameters
        
- __cutoff__ (*float, optional*) Radial cutoff distance for identifying neighbors.  Must be given if model is not given.
- __model__ (*str or file-like object, optional*) Gives the file path or content to load.  If given, no other parameters are allowed.
- __initialsize__ (*int, optional*) The number of neighbor positions to initially assign to each atom.  Default value is 20.
- __deltasize__ (*int, optional*) Specifies the number of extra neighbor positions to allow each atom when the number of neighbors exceeds the underlying array size.  Default value is 10.
            
Returns
        
- (*atomman.NeighborList*) The compiled list of neighbors.

In [8]:
# Identify neighbors within 3 angstroms
neighbors = system.neighborlist(cutoff=3)

The coordinataion numbers for the atoms can be retrieved with coord.

In [9]:
# Show average atomic coordination
print('Average coordination =', neighbors.coord.mean())

Average coordination = 4.0


Index selection on the NeighborList object will return the indices of the neighbor atoms.

In [10]:
# List neighbor atoms of atom 6
print('Neighbors of atom 6 =', neighbors[6])

Neighbors of atom 6 = [ 2 11 33 40]


The neighbor indices can then be used to filter other properties to only focus on an atom's neighbors.

In [11]:
# List the dmag values between atom 9 and its neighbors
print(system.dmag(9, neighbors[9]))

[2.35169198 2.35169198 2.35169198 2.35169198]


## 4. Region selectors<a id='section4'></a>

*Added version 1.3.0*

A number of geometric shape definitions are available in the atomman.region submodule to help identify regions in space above/below planes or inside/outside of regions. These are useful for constructing systems by slicing away atoms to create nanostructures, or for performing analysis on only select regions.

See the [3.3. Region selectors Jupyter Notebook](3.3. Region selectors.ipynb) for more details and a list of all available shapes.


In [12]:
# Define a plane normal to the y axis and positioned halfway across system
plane = am.region.Plane([0,1,0], system.box.bvect / 2)

# Count number of atoms in system, and above/below plane
print(f'{system.natoms} atoms in system')

abovecount = np.sum(plane.above(system.atoms.pos))
print(f'{abovecount} atoms above plane')

belowcount = np.sum(plane.below(system.atoms.pos))
print(f'{belowcount} atoms below plane')

# Define a sphere centered at [0,0,0] with radius = 6
sphere = am.region.Sphere([0,0,0], 6)

# Count atoms inside sphere
insidecount = np.sum(sphere.inside(system.atoms.pos))
print(f'{insidecount} atoms inside sphere')

64 atoms in system
24 atoms above plane
40 atoms below plane
11 atoms inside sphere


## 5. Basic tools<a id='section5'></a>

This lists some of the other basic tools and features in atomman.

### 5.1. Atomic information

- __atomman.tools.atomic_number()__ returns the atomic number associated with an element's atomic symbol.  
- __atomman.tools.atomic_symbol()__ returns the elemental symbol associated with an given atomic number.
- __atomman.tools.atomic_mass()__ returns the atomic mass of an element or isotope. The atom can be identified with atomic number or atomic/isotope symbol.

In [13]:
# Get atomic number for an atomic symbol
num = am.tools.atomic_number('Fe')
print(num)

# Get atomic symbol for an atomic number
symbol = am.tools.atomic_symbol(num)
print(symbol)

# Get atomic mass for an atomic symbol
mass = am.tools.atomic_mass(symbol)
print(mass)

# Get atomic mass for an atomic number
mass = am.tools.atomic_mass(num)
print(mass)

26
Fe
55.845
55.845


In [14]:
# Get atomic mass for an isotope
mass = am.tools.atomic_mass('Al-26')
print(mass)

25.986891904


### 5.2. axes_check()

The axes_check() function is useful when working in Cartesian systems. Given a (3,3) array representing three 3D Cartesian vectors:

- The three vectors are checked that they are orthogonal and right-handed.
- The corresponding array of unit vectors are returned. This can then be used for crystal transformations.

In [15]:
axes = [[-1, 0, 1],
        [ 1, 0, 1],
        [ 0, 1, 0]]
print(am.tools.axes_check(axes))

[[-0.70710678  0.          0.70710678]
 [ 0.70710678  0.          0.70710678]
 [ 0.          1.          0.        ]]


### 5.3. filltemplate()

The filltemplate() function takes a template and fills in values for delimited template variables.

In [16]:
madlibs = "My friend <name> really likes to use templates to <verb>, says that they are <adjective>!"
s_delimiter = '<'
e_delimiter = '>'

terms = {}
terms['name'] = 'Charlie'
terms['verb'] = 'program'
terms['adjective'] = 'delicious'

print(am.tools.filltemplate(madlibs, terms, s_delimiter, e_delimiter))

My friend Charlie really likes to use templates to program, says that they are delicious!


### 5.4. indexstr()

Iterates through all indicies of an array with a given shape, returning both the numeric index and a string representation.

In [17]:
for index, istr in am.tools.indexstr((3,2)):
    print('index ->', repr(index), ', istr ->', repr(istr))

index -> (0, 0) , istr -> '[0][0]'
index -> (0, 1) , istr -> '[0][1]'
index -> (1, 0) , istr -> '[1][0]'
index -> (1, 1) , istr -> '[1][1]'
index -> (2, 0) , istr -> '[2][0]'
index -> (2, 1) , istr -> '[2][1]'


### 5.5. uber_open_rmode

uber_open_rmode is a context manager that allows for similar reading of content from a file or from a string variable. It equivalently handles:
    
- str path name to a file
- str content
- open file-like object

In [18]:
# Define str and save to file
text = 'Here I am, read me!'
fname = 'text.txt'
with open(fname, 'w') as f:
    f.write(text)

# Use uber_open_rmode on text
with am.tools.uber_open_rmode(text) as f:
    print(f.read())
    
# Use uber_open_rmode on file path
with am.tools.uber_open_rmode(fname) as f:
    print(f.read())
    
# Use uber_open_rmode on file-like object
with open(fname, 'rb') as fobject:
    with am.tools.uber_open_rmode(fobject) as f:
        print(f.read())

b'Here I am, read me!'
b'Here I am, read me!'
b'Here I am, read me!'


### 5.6. vect_angle()

The vect_angle() function returns the angle between two vectors.

In [19]:
vect1 = 2*np.random.rand(3)-1
vect2 = 2*np.random.rand(3)-1

print('Angle between', vect1, 'and', vect2, '=')
print(am.tools.vect_angle(vect1, vect2), 'degrees')
print(am.tools.vect_angle(vect1, vect2, 'radian'), 'radians')

Angle between [ 0.14639682 -0.35639113 -0.23336601] and [ 0.16920961  0.1127633  -0.7395638 ] =
62.94056470473698 degrees
1.098520087162193 radians


### 5.7. duplicates_allclose()

Determine duplicates in dataframe based on tolerances.  The implementation
first uses pandas.DataFrame.duplicated on the `dcols` argument with
`keep=False` to keep all duplicates.  The duplicate sub-dataframe is then
sorted on both `dcols` and `fcols`.  A diff between each row is then done
on the sorted duplicates dataframe.  The float values are then checked for
their tolerances.

Note: False duplicates may be identified if tolerance ranges overlap.
Consider dataframe with rows 1,2,3.  If row 2 matches row 1 within the
tolerances, and row 3 matches row 2 within the tolerances, both rows 2 and
3 will be labeled as tolerances even if row 3 does not match row 1 within
the tolerances.

Parameters
- __dataframe__ (*pandas.DataFrame*) The dataframe to search for duplicates
- __dcols__ (*list*) The column names that are tested for exact duplicates.
- __fcols__ (*dict*) The column names (keys) that are tested using absolute tolerances (values).

Returns
- (*list of bool of length nrows*) False for first occurrence of checked values, True for subsequent duplicates.

In [20]:
# Generate test DataFrame
df = pd.DataFrame({'A':[1.00001, 1.00002, 3.0000, 1.000000], 'B':['Same', 'Diff', 'Same', 'Same']})
df

,A,B
0,1.00001,Same
1,1.00002,Diff
2,3.00000,Same
3,1.00000,Same


In [21]:
# Show unique values 
df[~am.tools.duplicates_allclose(df, dcols=['B'], fcols={'A':1e-4})]

/home/lmh1/Python-packages/atomman/atomman/tools/duplicates_allclose.py:217: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataframe.assign(duplicates=func(dataframe)).duplicates.fillna(False).rename()


,A,B
1,1.00002,Diff
2,3.00000,Same
3,1.00000,Same


### 5.8. Miller index conversions

- __atomman.tools.miller.vector3to4(indices)__ converts vectors from three-term Miller indices to four-term Miller-Bravais indices for hexagonal systems. *Updated version 1.2.6*: now returns vectors with magnitudes consistent withe given vectors rather than rescaling to the smallest integer representations.
- __atomman.tools.miller.vector4to3(indices)__ converts vectors from four-term Miller-Bravais indices to three-term Miller indices. *Updated version 1.2.6*: now returns vectors with magnitudes consistent withe given vectors rather than rescaling to the smallest integer representations.
- __atomman.tools.miller.plane3to4(indices)__ converts planes from three-term Miller indices to four-term Miller-Bravais indices for hexagonal systems. *Added version 1.2.8*
- __atomman.tools.miller.plane4to3(indices)__ converts planes from four-term Miller-Bravais indices to three-term Miller indices. *Added version 1.2.8*
- __atomman.tools.miller.vector_crystal_to_cartesian(indices, box)__ converts Miller and Miller-Bravais indices to Cartesian vectors based on a supplied box. *Updated version 1.2.6* renamed from vectortocartesian for consistency.
- __atomman.tools.miller.plane_crystal_to_cartesian(indices, box)__ converts Miller and Miller-Bravais plane indices to Cartesian normal vectors based on a supplied box. The method uses the definition of the crystal planes to identify two in-plane crystal vectors, converts them to Cartesian, and obtains the plane normal as the cross product of the two vectors. *Added version 1.3.2* 
- __atomman.tools.miller.vector_primitive_to_conventional(indices, setting)__ converts vectors relative to a primitive unit cell to a conventional unit cell in the given setting (p, a, b, c, i, f, t1 or t2). *Added version 1.2.6* *Updated version 1.4.10* to include t1 and t2 options.
- __atomman.tools.miller.vector_conventional_to_primitive(indices, setting)__ converts vectors relative to a conventional unit cell in the given setting (p, a, b, c, i, f, t1 or t2) to a primitive unit cell. *Added version 1.2.6* *Updated version 1.4.10* to include t1 and t2 options.
- __atomman.tools.miller.fromstring(str)__ reads a string representation of a Miller(-Bravais) vector/plane and converts it into a numpy array. *Added version 1.4.10*

In [22]:
# Test single vector conversions
print(am.tools.miller.vector3to4(np.array([3,3,3])))
print(am.tools.miller.vector4to3(np.array([1,1,-2,0])))

[ 1.  1. -2.  3.]
[3. 3. 0.]


In [23]:
# Generate random uvw crystal indices
indices = np.random.randint(-5,6, (3,3))
print(indices)
print()

# Convert to hexagonal uvtw's
indices = am.tools.miller.vector3to4(indices)
print(indices)
print()

# Convert back to uvw's and see that values are recovered
indices = am.tools.miller.vector4to3(indices)
print(indices)

[[ 5 -1 -1]
 [-5  1  4]
 [ 1  2 -2]]

[[ 3.66666667 -2.33333333 -1.33333333 -1.        ]
 [-3.66666667  2.33333333  1.33333333  4.        ]
 [ 0.          1.         -1.         -2.        ]]

[[ 5. -1. -1.]
 [-5.  1.  4.]
 [ 1.  2. -2.]]


In [24]:
# Test single plane conversions
print(am.tools.miller.plane3to4(np.array([3,3,3])))
print(am.tools.miller.plane4to3(np.array([1,1,-2,0])))

[ 3.  3. -6.  3.]
[1. 1. 0.]


In [25]:
# Generate random hkl crystal indices
indices = np.random.randint(-5,6, (3,3))
print(indices)
print()

# Convert to hexagonal hkil's
indices = am.tools.miller.plane3to4(indices)
print(indices)
print()

# Convert back to hkl's and see that values are recovered
indices = am.tools.miller.plane4to3(indices)
print(indices)

[[-1 -5 -1]
 [-3  0 -2]
 [-5 -4  2]]

[[-1. -5.  6. -1.]
 [-3.  0.  3. -2.]
 [-5. -4.  9.  2.]]

[[-1. -5. -1.]
 [-3.  0. -2.]
 [-5. -4.  2.]]


In [26]:
# Define a hexagonal box
a = uc.set_in_units(2.51, 'angstrom')
c = uc.set_in_units(4.07, 'angstrom')
box = am.Box(a=a, b=a, c=c, gamma=120)

# Pass Miller indices
indices = [[1,0,0],
           [0,1,0],
           [0,0,1]]
print(am.tools.miller.vector_crystal_to_cartesian(indices, box))
print()

# Pass equivalent Miller-Bravais indices
indices = [[ 2/3,-1/3,-1/3, 0],
           [-1/3, 2/3,-1/3, 0],
           [   0,   0,   0, 1]]
print(am.tools.miller.vector_crystal_to_cartesian(indices, box))

[[ 2.51        0.          0.        ]
 [-1.255       2.17372376  0.        ]
 [ 0.          0.          4.07      ]]

[[ 2.51        0.          0.        ]
 [-1.255       2.17372376  0.        ]
 [ 0.          0.          4.07      ]]


In [27]:
# Define a hexagonal box
a = uc.set_in_units(2.51, 'angstrom')
c = uc.set_in_units(4.07, 'angstrom')
box = am.Box(a=a, b=a, c=c, gamma=120)

# Pass Miller plane indices
indices = [ 0, 0, 1]
print(am.tools.miller.plane_crystal_to_cartesian(indices, box))
indices = [ 1, 0, 0]
print(am.tools.miller.plane_crystal_to_cartesian(indices, box))
print()

# Pass equivalent Miller-Bravais indices
indices = [ 0, 0, 0, 1]
print(am.tools.miller.plane_crystal_to_cartesian(indices, box))
indices = [ 1, 0,-1, 0]
print(am.tools.miller.plane_crystal_to_cartesian(indices, box))

[ 0. -0.  1.]
[ 0.8660254  0.5       -0.       ]

[ 0. -0.  1.]
[ 0.8660254  0.5       -0.       ]


In [28]:
# Define a primitive bcc unit cell box
a = uc.set_in_units(2.86, 'angstrom')
p_box = am.Box.trigonal(a * 3**0.5 / 2, alpha=109.466666667)
p_ucell = am.System(box=p_box)
print(p_ucell)

avect =  [ 2.477,  0.000,  0.000]
bvect =  [-0.825,  2.335,  0.000]
cvect =  [-0.825, -1.167,  2.023]
origin = [ 0.000,  0.000,  0.000]
natoms = 1
natypes = 1
symbols = (None,)
pbc = [ True  True  True]
per-atom properties = ['atype', 'pos']
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000


In [29]:
# Convert conventional box vectors to primitive vectors
a_uvw = am.tools.miller.vector_conventional_to_primitive([1, 0, 0], setting='i')
b_uvw = am.tools.miller.vector_conventional_to_primitive([0, 1, 0], setting='i')
c_uvw = am.tools.miller.vector_conventional_to_primitive([0, 0, 1], setting='i')
p_uvws = np.array([a_uvw, b_uvw, c_uvw])
print('primitive uvws:')
print(p_uvws)

# Convert back to conventional just for consistency
print('conventional uvws:')
print(am.tools.miller.vector_primitive_to_conventional(p_uvws, setting='i'))

primitive uvws:
[[ 0. -1. -1.]
 [ 1.  1.  0.]
 [ 1.  0.  1.]]
conventional uvws:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [30]:
# rotate system using p_uvws to get conventional unit cell
c_ucell = p_ucell.rotate(p_uvws)
print(c_ucell)

avect =  [ 2.860,  0.000,  0.000]
bvect =  [-0.000,  2.860,  0.000]
cvect =  [-0.000,  0.000,  2.860]
origin = [ 0.000,  0.000,  0.000]
natoms = 2
natypes = 1
symbols = (None,)
pbc = [ True  True  True]
per-atom properties = ['atype', 'pos']
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       1 |   1.430 |   1.430 |   1.430


In [31]:
# Read in string representations of the Miller(-Bravais) vectors and planes
miller_strings = [
    '1/2 [1 1 0]',
    '1/3 <1 1 -2 0>',
    '{1 1 1}',
    '(0 0 0 1)',
    '1 4 1'
]
for miller_str in miller_strings:
    print(f'{repr(miller_str):16} -> {repr(am.tools.miller.fromstring(miller_str))}')

'1/2 [1 1 0]'    -> array([0.5, 0.5, 0. ])
'1/3 <1 1 -2 0>' -> array([ 0.33333333,  0.33333333, -0.66666667,  0.        ])
'{1 1 1}'        -> array([1., 1., 1.])
'(0 0 0 1)'      -> array([0., 0., 0., 1.])
'1 4 1'          -> array([1., 4., 1.])


### 5.9. Crystal lattice identification

*Note version 1.4.4* These have been added as methods of the Box class.  These stand-alone functions are likely to be depreciated in the next major atomman version.

There are also a few tests for identifying if a supplied box is consistent with a standard representation of a crystal family unit cell.

- __atomman.tools.identifyfamily(box)__ returns str crystal family if box corresponds to a standard crystal representation. Otherwise, returns None.
- __atomman.tools.iscubic(box))__ returns bool indicating if box is a standard cubic box.
- __atomman.tools.ishexagonal(box))__ returns bool indicating if box is a standard hexagonal box.
- __atomman.tools.istetragonal(box))__ returns bool indicating if box is a standard tetragonal box.
- __atomman.tools.isrhombohedral(box))__ returns bool indicating if box is a standard rhombohedral box.
- __atomman.tools.isorthorhombic(box))__ returns bool indicating if box is a standard orthorhombic box.
- __atomman.tools.ismonoclinic(box))__ returns bool indicating if box is a standard monoclinic box.
- __atomman.tools.istriclinic(box))__ returns bool indicating if box is a standard triclinic box.

All of these functions use the following standard representation criteria:

- cubic: 
    - $a = b = c$
    - $\alpha = \beta = \gamma = 90$
- hexagonal: 
    - $a = b \ne c$
    - $\alpha = \beta = 90$
    - $\gamma = 120$
- tetragonal: 
    - $a = b \ne c$
    - $\alpha = \beta = \gamma = 90$
- rhombohedral:
    - $a = b = c$
    - $\alpha = \beta = \gamma \ne 90$
- orthorhombic: 
    - $a \ne b \ne c$
    - $\alpha = \beta = \gamma = 90$
- monoclinic: 
    - $a \ne b \ne c$
    - $\alpha = \gamma = 90$
    - $\beta \ne 90$
- triclinic: 
    - $a \ne b \ne c$
    - $\alpha \ne \beta \ne \gamma$

In [32]:
# Define an orthogonal box
a = uc.set_in_units(2.51, 'angstrom')
b = uc.set_in_units(3.13, 'angstrom')
c = uc.set_in_units(4.07, 'angstrom')
box = am.Box(a=a, b=b, c=c)

print('identifyfamily =', am.tools.identifyfamily(box))
print('iscubic =       ', am.tools.iscubic(box))
print('ishexagonal =   ', am.tools.ishexagonal(box))
print('istetragonal =  ', am.tools.istetragonal(box))
print('isrhombohedral =', am.tools.isrhombohedral(box))
print('isorthorhombic =', am.tools.isorthorhombic(box))
print('ismonoclinic =  ', am.tools.ismonoclinic(box))
print('istriclinic =   ', am.tools.istriclinic(box))

identifyfamily = orthorhombic
iscubic =        False
ishexagonal =    False
istetragonal =   False
isrhombohedral = False
isorthorhombic = True
ismonoclinic =   False
istriclinic =    False


In [33]:
# Define a non-standard tetragonal box with a=c!=b
box = am.Box(a=a, b=b, c=a)
print('identifyfamily =', am.tools.identifyfamily(box))

identifyfamily = None


### 5.10. compositionstr()

*Added version 1.2.7*

Takes a list of symbols and the counts for each and returns a reduced composition string.  Used by System.composition.

In [34]:
symbols = ['Si', 'Al', 'Si']
counts = [500, 1000, 2000]
print('Composition =', am.tools.compositionstr(symbols, counts))

Composition = Al2Si5


**File Cleanup**

In [35]:
os.remove('text.txt')